In [1]:
# Load packages
import numpy as np
import pandas as pd
import shared_functions as sf
from category_encoders import OrdinalEncoder

In [2]:
# Define file name
file_name = 'property-sales_new-york-city_2022'

In [3]:
# Load data
df = pd.read_parquet(f'../data/raw/{file_name}_geocoded.parquet')
data_overview = pd.read_csv(f'../data/raw/{file_name}_geocoded_data-overview.csv', index_col=0)
cpi = pd.read_csv('../data/raw/consumer-price-index_us_1982-84_2022.csv', index_col=0, parse_dates=True)

In [4]:
# Use most recent pandas data types (e.g. pd.NA)
df = df.convert_dtypes()
data_overview = data_overview.convert_dtypes()
cpi = cpi.convert_dtypes()
cpi.index = cpi.index.to_period('M')

In [5]:
# Get dictionary of columns for each variable type
variable_types = {variable_type: data_overview.column[data_overview.variable_type == variable_type].to_list() for variable_type in data_overview.variable_type.unique()}

In [6]:
# Show data overview
data_overview

,column,dtype,n_unique,n_missing,variable_type
n,,,,,
0,borough,Int64,5,0,categorical
1,neighborhood,string,252,0,categorical
2,building_class_category,string,44,0,categorical
3,tax_class_at_present,string,10,112,categorical
4,block,Int64,11742,0,categorical
5,lot,Int64,2647,0,categorical
6,easement,Int64,0,93427,categorical
7,building_class_at_present,string,157,112,categorical
8,address,string,90867,0,categorical


In [7]:
# Drop easement column since it is empty
df.drop(columns='easement', inplace=True)
variable_types['categorical'].remove('easement')

In [8]:
# Drop entries without coordinates to ensure comparability acorss models
print(f'Number of rows before: {len(df)}')
df.dropna(subset=['location_lat', 'location_long'], inplace=True)
print(f'Number of rows after: {len(df)}')

Number of rows before: 93427
Number of rows after: 85901


In [9]:
# Drop entries of sales for USD 0
print(f'Number of rows before: {len(df)}')
df = df[df.sale_price != 0]
print(f'Number of rows after: {len(df)}')

Number of rows before: 85901
Number of rows after: 62202


In [10]:
# Drop outliers
print(f'Number of rows before: {len(df)}')
print(f'Price range before: {min(df.sale_price)} - {max(df.sale_price)}')
q_low = df.sale_price.quantile(0.025)
q_hi  = df.sale_price.quantile(0.975)
df = df[(df.sale_price < q_hi) & (df.sale_price > q_low)]
print(f'Number of rows after: {len(df)}')
print(f'Price range after: {min(df.sale_price)} - {max(df.sale_price)}')

Number of rows before: 62202
Price range before: 1 - 931000000
Number of rows after: 59046
Price range after: 35000 - 8995000


In [11]:
# Remove apartment number from address column
df.address = df.address.str.split(',').str[0]

In [12]:
# Extract additional features from selected categorical variables
df['tax_class_at_present_prefix'] = df.tax_class_at_present.str.extract('(\d)')
variable_types['categorical'].append('tax_class_at_present_prefix')

df['building_class_at_present_prefix'] = df.building_class_at_present.str.extract('(.)')
variable_types['categorical'].append('building_class_at_present_prefix')

df['street_name'] = df.address.str.extract('^\d+\s(.*)')
variable_types['categorical'].append('street_name')

df['building_class_at_time_of_sale_prefix'] = df.building_class_at_time_of_sale.str.extract('(.)')
variable_types['categorical'].append('building_class_at_time_of_sale_prefix')

In [13]:
# Calculate the inflation rate for each month
cpi_monthly = cpi.copy()
cpi_monthly['inflation_rate'] = cpi_monthly.cpi / cpi_monthly.cpi.iloc[0]

In [14]:
# Obtain adjusted sale price for each sample
sale_price_adj = df.loc[:, ['sale_date', 'sale_price']]
sale_price_adj['sale_month'] = sale_price_adj.sale_date.dt.to_period('M')
sale_price_adj = sale_price_adj.join(cpi_monthly[['inflation_rate']], on='sale_month')
sale_price_adj = sale_price_adj.sale_price / sale_price_adj.inflation_rate

In [15]:
# Replace sale price with adjusted sale price
df['sale_price_adj'] = sale_price_adj
variable_types['numerical'].append('sale_price_adj')

In [16]:
# Convert dates into timestamps
df[variable_types['date']] = df[variable_types['date']].astype(int) / 10**9
variable_types['numerical'].extend(variable_types['date'])
variable_types['date'] = [column for column in variable_types['date'] if column not in variable_types['date']]

In [17]:
# Use most recent pandas data types (e.g. pd.NA)
df = df.convert_dtypes()

In [18]:
# Save pre-processed data
df.to_parquet(f'../data/processed/{file_name}_pre-processed.parquet')
df.to_csv(f'../data/processed/{file_name}_pre-processed.csv')

In [19]:
# Generate new data overview
data_overview = sf.data_overview(df)

In [20]:
# Add variable type to new data overview
for variable_type, columns in variable_types.items():
    for column in columns:
        data_overview.loc[data_overview.column == column, 'variable_type'] = variable_type

In [21]:
# Show new data overview
data_overview

,column,dtype,n_unique,n_missing,variable_type
n,,,,,
0,borough,Int64,5,0,categorical
1,neighborhood,string,251,0,categorical
2,building_class_category,string,43,0,categorical
3,tax_class_at_present,string,10,93,categorical
4,block,Int64,9950,0,categorical
5,lot,Int64,2268,0,categorical
6,building_class_at_present,string,134,93,categorical
7,address,string,36993,0,categorical
8,apartment_number,string,3514,42280,categorical


In [22]:
# Save new data overview
data_overview.to_csv(f'../data/processed/{file_name}_pre-processed_data-overview.csv')

In [23]:
# Subset the index
subset_index, subset_keys = sf.subset_index(df.index, test_size=0.2, val_size=.1,
                                            save_index_as=f'../data/processed/subset_index.csv',
                                            save_keys_as=f'../data/processed/subset_keys.csv',
                                            save_sizes_as=f'../data/processed/subset_sizes.csv'
                                            )

Subset  Size (n)  Size (%)
--------------------------
train      41333    70.00%
val         5904    10.00%
test       11809    20.00%
--------------------------
total      59046   100.00%
